<h1 style = "text-decoration: underline">TOTAL PORTFOLIO ANALYSIS</h1>


<h2 style = "text-decoration: underline">Importações</h2>


In [ ]:
# IMPORTS
import pandas as pd
from pandas_datareader import data as web
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import skew
from scipy.stats import kurtosis
        #In SciPy the kurtosis function is actually computing excess kurtosis (greater than 0 suggests non-normality). 
        #If you wanted to calculate the true sample kurtosis, you would actually need to add 3 to the result. 
        #For most cases, you're going to be interested in excess kurtosis anyways, so this functionality 
        #is fine as long as you are aware of it. 
        #In finance, high excess kurtosis is an indication of high risk. 
        #When large movements in returns happen often, this can be a very bad thing for your portfolio if it moves in 
        #the wrong direction. High kurtosis distributions are said to have "thick tails", which means that outliers, 
        #such as extreme negative and positive returns, are more common.
from scipy import stats
from scipy.stats import norm
import scipy.optimize as sco
import seaborn as sns
import win32com.client
from openpyxl import load_workbook
from itertools import islice
from tqdm import tqdm
import time
import datetime as dt

plt.style.use('classic')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<h2 style = "text-decoration: underline">Obter dados do Portfolio</h2>


In [ ]:
##Pimeiramente abra a planilha Portfolio_Acoes e inira os ativos para importar do Yahoo Finance - Planilha "990 Portfolio_Acoes"
#Ações brasileiras devem ser inseridas com .SA após o símbolo
df = pd.read_excel('990 Portfolio_Acoes.xlsx')
ativos = df['AÇÕES'].tolist()
num_acoes = len(ativos)
num_acoes = int(num_acoes)

print(ativos, '\n')

In [ ]:
##Importar do Yahoo Finance
data_final = dt.datetime.today()
data_inicial = data_final-dt.timedelta(days=1825)

ls_key = 'Adj Close'
start = data_inicial
end = data_final   
f = web.DataReader(ativos, 'yahoo',start,end)

dados = f[ls_key]
dados_df = pd.DataFrame(dados).dropna()

dados_df.to_csv('999 Dados.csv', sep = ';', index = True, header = True)
dados = pd.read_csv('999 Dados.csv', sep = ';', parse_dates=['Date'])

print (dados)

In [ ]:
## Abra a planilha Portfolio_Fundos e insira os Fundos ou outros Ativos para importar de outras fontes, via planilha Excel (Criar duas planilhas, uma para obter os dados
#da fonte, outra para padronizar para importação, as datas são as obtidas acima, a coluna com os preços é a segunda [1], etc.)

##Inserir os Fundos para importar para o Python - Planilha "990 Portfolio_Fundos"

df = pd.read_excel('990 Portfolio_Fundos.xlsx')
fundos = df['FUNDOS'].tolist()
num_fundos = len(fundos)
num_fundos = int(num_fundos)

n = 0
for x in range(num_fundos): 
    abrir = fundos[n]
    xl = win32com.client.DispatchEx("Excel.Application")
    wb2 = xl.workbooks.open(r'...\999 Dados.csv')
    time.sleep(10)
    wb1 = xl.workbooks.open(r'...\990 '+ abrir +'.xlsx') 
                                                                                            #planilha para obter dados
    time.sleep(25)
    wb = xl.workbooks.open(r'...\990 '+ abrir +
                           ' - Dados Python.xlsx') #planilha padronizada para importação pelo Python
    time.sleep(25)
    wb.Close(True)
    wb1.Close(True)
    wb2.Close(True)
    time.sleep(10)
    read_file = pd.read_excel (r'...\990 '+ abrir +
                               ' - Dados Python.xlsx', index_col = 'Date')
    read_file.to_csv (r'...\999 '+ abrir +
                      ' - Dados Python.csv', sep = ';', index = True, header=True)
    time.sleep(10)
    temp = pd.read_csv('999 '+ abrir +' - Dados Python.csv', sep = ';', usecols=[1])
    xl.Quit() #fechar o Excel definitivamente
    xl = None #fechar o Excel definitivamente
    dados = pd.concat([dados,temp],axis=1).reindex(dados.index) #concatenar os DataFrame
    n += 1

dados.to_csv('999 Dados.csv', sep = ';', index = None, header = True)
dados = pd.read_csv('999 Dados.csv', sep = ';', index_col = 0, header = 0)
print (dados)


<h2 style = "text-decoration: underline">Primeiros cálculos do Portfolio</h2>


In [ ]:
#RETORNO PERCENTUAL DIARIO
returns = dados.pct_change(fill_method='ffill')

plt.figure(figsize=(14, 7))
for c in returns.columns.values:
    plt.plot(returns.index, returns[c], lw=3, alpha=0.8,label=c)
plt.legend(loc='upper right', fontsize=8)
plt.ylabel('daily returns')

# Calculate the average daily return of the stock
mean_return_daily = np.mean(returns)
print ("1. retorno médio diario em %: ", mean_return_daily*100)

# Calculate the implied annualized average return
mean_return_annualized = ((1+mean_return_daily)**252)-1
print("2. retorno médio anualizado em %: ", mean_return_annualized*100)

# Calculate the standard deviation of daily return of the stock
sigma_daily = np.std(returns)
print("3. volatilidade média diaria em %: ", sigma_daily*100)

# Annualize the standard deviation
sigma_annualized = sigma_daily*np.sqrt(252)
print("4. volatilidade média anualizada em %: ", sigma_annualized*100)

# Calculate the daily variance
variance_daily = sigma_daily**2
print("5. variância média diario em %: ", variance_daily*100)

# Calculate the annualized variance
variance_annualized = sigma_annualized**2
print("6. variância média anualizado em %: ", variance_annualized)

# Calculate the third moment (skewness) of the returns distribution
returns_skewness = skew(returns.dropna())
print("7. distribuição normal (skew ou assimetria): ", returns_skewness)

# Calculate the excess kurtosis of the returns distribution
excess_kurtosis = kurtosis(returns.dropna())
print("8. distribuição normal (kurtosis ou kurtose): ", excess_kurtosis)

# Derive the true fourth moment of the returns distribution
fourth_moment = excess_kurtosis+3
print("> real kurtosis: ", fourth_moment)#In SciPy the kurtosis function is actually computing excess kurtosis. 
                    #If you wanted to calculate the true sample kurtosis, you would actually need to add 3 to the result.
    
p_value = stats.shapiro(returns.dropna())[1]
if p_value <= 0.05:
    print("9. Teste de normalidade: ", p_value, ". Dados com distribuição não normal")
else:
    print("9. Teste de normalidade: ", p_value, ". Dados com distribuição normal")

<h2 style = "text-decoration: underline">Cálculos para análise do Portfolio</h2>

#O código dessa seção foi obtido e adaptado de: https://towardsdatascience.com/efficient-frontier-portfolio-optimisation-in-python-e7844051e7f


In [ ]:
# Variáveis para construção dos Portfolio Automáticos e análises diversas
np.random.seed(0)
mean_returns = returns.mean()
cov_matrix = returns.cov()
corr_matrix=returns.corr()
num_portfolios = 100000
risk_free_rate = 0.1150

corr_matrix.to_csv('993 CORRELAÇÃO.csv', sep = ';', index = True, header=True)

In [ ]:
# Variáveis para construção dos Portfolio Automáticos e análises diversas
np.random.seed(0)
num_ativos = num_acoes + num_fundos
num_ativos = int(num_ativos)
#construção do retorno anualizado
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

#construção do portfolio aleatório
def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios))
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(num_ativos)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

#construção do portfolio sem ativos SHARPE negativo
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

#construção do portfolio de MAXIMO SHARPE sem ativos com SHARPE negativo
def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

#construção do retorno anualizado
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

#construção da VOLATILIDADE
def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

#construção do Retorno Eficiente
def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

#construção da FRONTEIRA EFICIENTE
def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

In [ ]:
np.random.seed(0)
def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe_idx = np.argmax(results[2])
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=dados.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation.to_csv('991 MAX SHARPE.csv', sep = ';', index = True, header=True)
    max_sharpe_allocation.to_csv('993 My PORTFOLIO.csv', sep = ';', index = True, header=True)
    w1 = max_sharpe_allocation
    max_sharpe_allocation = max_sharpe_allocation.T
        
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=dados.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation.to_csv('991 MIN VOLAT.csv', sep = ';', index = True, header=True)
    min_vol_allocation = min_vol_allocation.T
    
    an_vol = np.std(returns) * np.sqrt(252)
    an_rt = mean_returns * 252
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Aleatório\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Aleatório\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    print ("-"*80)
    print ("Individual Stock Returns and Volatility\n")
    for i, txt in enumerate(dados.columns):
        print (txt,":","annualised return",round(an_rt[i],2),", annualised volatility:",round(an_vol[i],2))
    print ("-"*80)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=300, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=300, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.01)

display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)

In [ ]:
def display_ef_with_selected(mean_returns, cov_matrix, risk_free_rate):
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=dados.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation.to_csv('992 MAX SHARPE.csv', sep = ';', index = True, header=True)
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=dados.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation.to_csv('992 MIN VOLAT.csv', sep = ';', index = True, header=True)
    min_vol_allocation = min_vol_allocation.T
    
    an_vol = np.std(returns) * np.sqrt(252)
    an_rt = mean_returns * 252
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Selecionado\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Selecionado\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    print ("-"*80)
    print ("Individual Stock Returns and Volatility\n")
    for i, txt in enumerate(dados.columns):
        print (txt,":","annualised return",round(an_rt[i],2),", annualised volatility:",round(an_vol[i],2))
    print ("-"*80)
    
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.scatter(an_vol,an_rt,marker='o',s=200)

    for i, txt in enumerate(dados.columns):
        ax.annotate(txt, (an_vol[i],an_rt[i]), xytext=(10,0), textcoords='offset points')
        
    ax.scatter(sdp,rp,marker='*',color='r',s=300, label='Maximum Sharpe ratio')
    ax.scatter(sdp_min,rp_min,marker='*',color='g',s=300, label='Minimum volatility')
    target = np.linspace(rp_min, 0.34, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    ax.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    ax.set_title('Portfolio Optimization with Individual Stocks')
    ax.set_xlabel('annualised volatility')
    ax.set_ylabel('annualised returns')
    ax.legend(labelspacing=0)
    
display_ef_with_selected(mean_returns, cov_matrix, risk_free_rate)

<h1 style = "text-decoration: underline">Cálculos de Análise de Portfolios e Value at Risk</h1>

#O código dessa seção foi obtido e adaptado de: https://ebookreading.net/view/book/EB9781787125698_143.html e de: https://www.interviewqs.com/blog/value-at-risk

<h3 style = "text-decoration: underline">Análise e Value at Risk PORTFOLIO ALEATÓRIO COM MAX SHARPE</h3>

In [ ]:
# Set the investment weights
df = pd.read_csv('991 MAX SHARPE.csv', sep = ';', index_col = None)
weights = df['allocation'].to_numpy()

# Set an initial investment level
initial_investment = 100000

# Retorno médio do portfolio
port_mean = mean_returns.dot(weights)

# Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
# Calculate mean of investment
mean_investment = (1+port_mean) * initial_investment
             
# Calculate standard deviation of investmnet
stdev_investment = initial_investment * port_stdev

# Calculate Índice de Sharpe e Volatilidade anualizada
isharpe = ((port_mean*252)-risk_free_rate)/(port_stdev*252)
print("Índice de Sharpe: ", np.round(isharpe,2),"\n")
print("Volatilidade anualizada: ", np.round(port_stdev*np.sqrt(252), 0),"%\n")

# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

# Using SciPy ppf method to generate values for the inverse cumulative distribution function to a normal distribution
# Plugging in the mean, standard deviation of our portfolio as calculated above

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
VaR_1d = initial_investment - cutoff1

# we will use return for average stock return of Portfolio
sample_size = returns.shape[0]
sample_mean = port_mean
sample_std = port_stdev / sample_size**0.5

# left and right quantile
z_left = norm.ppf(0.025)
z_right = norm.ppf(0.975)

# upper and lower bound
interval_left = sample_mean + z_left * sample_std
interval_right = sample_mean + z_right * sample_std

# 95% confidence interval tells you that there will be 95% chance that the average stock return lies between "interval_left"
# and "interval_right" em casos de distribuição normal, o que não se aplica a todos os casos.
# De qualquer forma, o resultado apresentado, embora não se possa dizer que tenha um intervalo de confiança de 95%, nos dá uma
# boa ilutração para análise do portfolio

print('Intervalo esperado do retorno diário: ', np.round((interval_left, interval_right),3), '%\n')

print('Value at Risk Calculado para um dia: ',np.round((VaR_1d/initial_investment),3),'%\n')

#VaR Monte Carlo Simulation
confidence = 0.95
nSimulations = 100000
sp.random.seed(0) 
ret2=sp.random.normal(port_mean,port_stdev,nSimulations) 
ret3=np.sort(ret2) 
m=int(nSimulations*(1-confidence))
VaR2=initial_investment*(ret3[m])
print ("Value at Risk Calculado Sim. de Monte Carlo: ",np.round((VaR2/initial_investment)*-1,3),"% \n")

# Calculate n Day VaR
var_array = []
num_days = int(30)
for x in range(1, num_days+1):    
    var_array.append(np.round(VaR_1d/initial_investment * np.sqrt(x),2))

print(str(x) + " day VaR Calculado: " + str(np.round(VaR_1d/initial_investment * np.sqrt(x),2)),"%\n")

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss esperado")
plt.title("Max portfolio loss esperado (VaR) over 30 days period")
plt.plot(var_array, "r")

<h3 style = "text-decoration: underline">Análise e Value at Risk PORTFOLIO ALEATÓRIO COM MIN VOLATILIDADE</h3>

In [ ]:
# Set the investment weights
df = pd.read_csv('991 MIN VOLAT.csv', sep = ';', index_col = None)
weights = df['allocation'].to_numpy()

# Set an initial investment level
initial_investment = 100000

# Retorno médio do portfolio
port_mean = mean_returns.dot(weights)

# Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
# Calculate mean of investment
mean_investment = (1+port_mean) * initial_investment
             
# Calculate standard deviation of investmnet
stdev_investment = initial_investment * port_stdev

# Calculate Índice de Sharpe e Volatilidade anualizada
isharpe = ((port_mean*252)-risk_free_rate)/(port_stdev*252)
print("Índice de Sharpe: ", np.round(isharpe,2),"\n")
print("Volatilidade anualizada: ", np.round(port_stdev*np.sqrt(252), 0),"%\n")

# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

# Using SciPy ppf method to generate values for the inverse cumulative distribution function to a normal distribution
# Plugging in the mean, standard deviation of our portfolio as calculated above

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
VaR_1d = initial_investment - cutoff1

# we will use return for average stock return of Portfolio
sample_size = returns.shape[0]
sample_mean = port_mean
sample_std = port_stdev / sample_size**0.5

# left and right quantile
z_left = norm.ppf(0.025)
z_right = norm.ppf(0.975)

# upper and lower bound
interval_left = sample_mean + z_left * sample_std
interval_right = sample_mean + z_right * sample_std

# 95% confidence interval tells you that there will be 95% chance that the average stock return lies between "interval_left"
# and "interval_right" em casos de distribuição normal, o que não se aplica a todos os casos.
# De qualquer forma, o resultado apresentado, embora não se possa dizer que tenha um intervalo de confiança de 95%, nos dá uma
# boa ilutração para análise do portfolio

print('Intervalo esperado do retorno diário: ', np.round((interval_left, interval_right),3), '%\n')

print('Value at Risk Calculado para um dia: ',np.round((VaR_1d/initial_investment),3),'%\n')

#VaR Monte Carlo Simulation
confidence = 0.95
nSimulations = 100000
sp.random.seed(0) 
ret2=sp.random.normal(port_mean,port_stdev,nSimulations) 
ret3=np.sort(ret2) 
m=int(nSimulations*(1-confidence))
VaR2=initial_investment*(ret3[m])
print ("Value at Risk Calculado Sim. de Monte Carlo: ",np.round((VaR2/initial_investment)*-1,3),"% \n")

# Calculate n Day VaR
var_array = []
num_days = int(30)
for x in range(1, num_days+1):    
    var_array.append(np.round(VaR_1d/initial_investment * np.sqrt(x),2))

print(str(x) + " day VaR Calculado: " + str(np.round(VaR_1d/initial_investment * np.sqrt(x),2)),"%\n")

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss esperado")
plt.title("Max portfolio loss esperado (VaR) over 30 days period")
plt.plot(var_array, "r")

<h3 style = "text-decoration: underline">Análise e Value at Risk PORTFOLIO SELECIONADO PARA MAX SHARPE</h3>

In [ ]:
# Set the investment weights
df = pd.read_csv('992 MAX SHARPE.csv', sep = ';', index_col = None)
weights = df['allocation'].to_numpy()

# Set an initial investment level
initial_investment = 100000

# Retorno médio do portfolio
port_mean = mean_returns.dot(weights)

# Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
# Calculate mean of investment
mean_investment = (1+port_mean) * initial_investment
             
# Calculate standard deviation of investmnet
stdev_investment = initial_investment * port_stdev

# Calculate Índice de Sharpe e Volatilidade anualizada
isharpe = ((port_mean*252)-risk_free_rate)/(port_stdev*252)
print("Índice de Sharpe: ", np.round(isharpe,2),"\n")
print("Volatilidade anualizada: ", np.round(port_stdev*np.sqrt(252), 0),"%\n")

# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

# Using SciPy ppf method to generate values for the inverse cumulative distribution function to a normal distribution
# Plugging in the mean, standard deviation of our portfolio as calculated above

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
VaR_1d = initial_investment - cutoff1

# we will use return for average stock return of Portfolio
sample_size = returns.shape[0]
sample_mean = port_mean
sample_std = port_stdev / sample_size**0.5

# left and right quantile
z_left = norm.ppf(0.025)
z_right = norm.ppf(0.975)

# upper and lower bound
interval_left = sample_mean + z_left * sample_std
interval_right = sample_mean + z_right * sample_std

# 95% confidence interval tells you that there will be 95% chance that the average stock return lies between "interval_left"
# and "interval_right" em casos de distribuição normal, o que não se aplica a todos os casos.
# De qualquer forma, o resultado apresentado, embora não se possa dizer que tenha um intervalo de confiança de 95%, nos dá uma
# boa ilutração para análise do portfolio

print('Intervalo esperado do retorno diário: ', np.round((interval_left, interval_right),3), '%\n')

print('Value at Risk Calculado para um dia: ',np.round((VaR_1d/initial_investment),3),'%\n')

#VaR Monte Carlo Simulation
confidence = 0.95
nSimulations = 100000
sp.random.seed(0) 
ret2=sp.random.normal(port_mean,port_stdev,nSimulations) 
ret3=np.sort(ret2) 
m=int(nSimulations*(1-confidence))
VaR2=initial_investment*(ret3[m])
print ("Value at Risk Calculado Sim. de Monte Carlo: ",np.round((VaR2/initial_investment)*-1,3),"% \n")

# Calculate n Day VaR
var_array = []
num_days = int(30)
for x in range(1, num_days+1):    
    var_array.append(np.round(VaR_1d/initial_investment * np.sqrt(x),2))

print(str(x) + " day VaR Calculado: " + str(np.round(VaR_1d/initial_investment * np.sqrt(x),2)),"%\n")

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss esperado")
plt.title("Max portfolio loss esperado (VaR) over 30 days period")
plt.plot(var_array, "r")

<h3 style = "text-decoration: underline">Análise e Value at Risk PORTFOLIO SELECIONADO PARA MIN VOLATILIDADE</h3>

In [ ]:
# Set the investment weights
df = pd.read_csv('992 MIN VOLAT.csv', sep = ';', index_col = None)
weights = df['allocation'].to_numpy()

# Set an initial investment level
initial_investment = 100000

# Retorno médio do portfolio
port_mean = mean_returns.dot(weights)

# Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
# Calculate mean of investment
mean_investment = (1+port_mean) * initial_investment
             
# Calculate standard deviation of investmnet
stdev_investment = initial_investment * port_stdev

# Calculate Índice de Sharpe e Volatilidade anualizada
isharpe = ((port_mean*252)-risk_free_rate)/(port_stdev*252)
print("Índice de Sharpe: ", np.round(isharpe,2),"\n")
print("Volatilidade anualizada: ", np.round(port_stdev*np.sqrt(252), 0),"%\n")

# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

# Using SciPy ppf method to generate values for the inverse cumulative distribution function to a normal distribution
# Plugging in the mean, standard deviation of our portfolio as calculated above

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
VaR_1d = initial_investment - cutoff1

# we will use return for average stock return of Portfolio
sample_size = returns.shape[0]
sample_mean = port_mean
sample_std = port_stdev / sample_size**0.5

# left and right quantile
z_left = norm.ppf(0.025)
z_right = norm.ppf(0.975)

# upper and lower bound
interval_left = sample_mean + z_left * sample_std
interval_right = sample_mean + z_right * sample_std

# 95% confidence interval tells you that there will be 95% chance that the average stock return lies between "interval_left"
# and "interval_right" em casos de distribuição normal, o que não se aplica a todos os casos.
# De qualquer forma, o resultado apresentado, embora não se possa dizer que tenha um intervalo de confiança de 95%, nos dá uma
# boa ilutração para análise do portfolio

print('Intervalo esperado do retorno diário: ', np.round((interval_left, interval_right),3), '%\n')

print('Value at Risk Calculado para um dia: ',np.round((VaR_1d/initial_investment),3),'%\n')

#VaR Monte Carlo Simulation
confidence = 0.95
nSimulations = 100000
sp.random.seed(0) 
ret2=sp.random.normal(port_mean,port_stdev,nSimulations) 
ret3=np.sort(ret2) 
m=int(nSimulations*(1-confidence))
VaR2=initial_investment*(ret3[m])
print ("Value at Risk Calculado Sim. de Monte Carlo: ",np.round((VaR2/initial_investment)*-1,3),"% \n")

# Calculate n Day VaR
var_array = []
num_days = int(30)
for x in range(1, num_days+1):    
    var_array.append(np.round(VaR_1d/initial_investment * np.sqrt(x),2))

print(str(x) + " day VaR Calculado: " + str(np.round(VaR_1d/initial_investment * np.sqrt(x),2)),"%\n")

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss esperado")
plt.title("Max portfolio loss esperado (VaR) over 30 days period")
plt.plot(var_array, "r")

<h3 style = "text-decoration: underline">Análise e Value at Risk PORTFOLIO ELABORADO</h3>

In [ ]:
# Set the investment weights na planilha MY PORTFOLIO, da forma que achar mais apropriado
xl = win32com.client.DispatchEx("Excel.Application")
wb1 = xl.workbooks.open(r'...\993 MY PORTFOLIO.csv')
xl.visible = True
continuar = 'N'
continuar = input("O arquivo já está fechado para podermos continuar? (S/N)")
while continuar != 'S':
    continuar = input("O arquivo já está fechado para podermos continuar? (S/N)")
xl.Quit() #fechar o Excel definitivamente
xl = None #fechar o Excel definitivamente
time.sleep(3)
df = pd.read_csv('993 MY PORTFOLIO.csv', sep = ';', index_col = None)
weights = df['allocation'].to_numpy()

# Set an initial investment level
initial_investment = 100000

# Retorno médio do portfolio
port_mean = mean_returns.dot(weights)

# Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
# Calculate mean of investment
mean_investment = (1+port_mean) * initial_investment
             
# Calculate standard deviation of investmnet
stdev_investment = initial_investment * port_stdev

# Calculate Índice de Sharpe e Volatilidade anualizada
isharpe = ((port_mean*252)-risk_free_rate)/(port_stdev*252)
print("Índice de Sharpe: ", np.round(isharpe,2),"\n")
print("Volatilidade anualizada: ", np.round(port_stdev*np.sqrt(252), 0),"%\n")

# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

# Using SciPy ppf method to generate values for the inverse cumulative distribution function to a normal distribution
# Plugging in the mean, standard deviation of our portfolio as calculated above

cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)

#Finally, we can calculate the VaR at our confidence interval
VaR_1d = initial_investment - cutoff1

# we will use return for average stock return of Portfolio
sample_size = returns.shape[0]
sample_mean = port_mean
sample_std = port_stdev / sample_size**0.5

# left and right quantile
z_left = norm.ppf(0.025)
z_right = norm.ppf(0.975)

# upper and lower bound
interval_left = sample_mean + z_left * sample_std
interval_right = sample_mean + z_right * sample_std

# 95% confidence interval tells you that there will be 95% chance that the average stock return lies between "interval_left"
# and "interval_right" em casos de distribuição normal, o que não se aplica a todos os casos.
# De qualquer forma, o resultado apresentado, embora não se possa dizer que tenha um intervalo de confiança de 95%, nos dá uma
# boa ilutração para análise do portfolio

print('Intervalo esperado do retorno diário: ', np.round((interval_left, interval_right),3), '%\n')

print('Value at Risk Calculado para um dia: ',np.round((VaR_1d/initial_investment),3),'%\n')

#VaR Monte Carlo Simulation
confidence = 0.95
nSimulations = 100000
sp.random.seed(0) 
ret2=sp.random.normal(port_mean,port_stdev,nSimulations) 
ret3=np.sort(ret2) 
m=int(nSimulations*(1-confidence))
VaR2=initial_investment*(ret3[m])
print ("Value at Risk Calculado Sim. de Monte Carlo: ",np.round((VaR2/initial_investment)*-1,3),"% \n")

# Calculate n Day VaR
var_array = []
num_days = int(30)
for x in range(1, num_days+1):    
    var_array.append(np.round(VaR_1d/initial_investment * np.sqrt(x),2))

print(str(x) + " day VaR Calculado: " + str(np.round(VaR_1d/initial_investment * np.sqrt(x),2)),"%\n")

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss esperado")
plt.title("Max portfolio loss esperado (VaR) over 30 days period")
plt.plot(var_array, "r")